### Create epochs based on expert annonations 
 - Geletaw Sahle Tegenaw(Ph.D), Norman Delanty (Prof), Tomas Ward (Prof)




In [1]:
import os
import mne
import pandas as pd
import numpy as np
from mne import Epochs
from typing import List, Tuple

In [2]:
def create_epochs_from_folder(folder_path, save_folder="/home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs"):
    """
    Create and save epochs from all .fif files in a folder based on annotations.

    Parameters:
        folder_path : str
            Path to the folder containing raw .fif files with annotations.
        save_folder : str
            Folder to save the processed epoch files.
    """
    # os.makedirs(save_folder, exist_ok=True)

    fif_files = [f for f in os.listdir(folder_path) if f.endswith(".fif")]

    if not fif_files:
        print("No .fif files found in the specified folder.")
        return

    for fif_file in fif_files:
        raw_filepath = os.path.join(folder_path, fif_file)
        print(f"Processing file: {fif_file}")

        try:
            raw = mne.io.read_raw_fif(raw_filepath, preload=True)

            # Create events from annotations
            event_id = {'S': 1, 'NS': 2, 'D': 3}
            events, event_id_map = mne.events_from_annotations(raw, event_id=event_id)

            # Epoch parameters
            sfreq = raw.info['sfreq']
            tmin = 0.0
            tmax = (sfreq - 1) / sfreq
            baseline = None

            # Create epochs
            epochs = mne.Epochs(raw, events=events, event_id=event_id,
                                tmin=tmin, tmax=tmax, baseline=baseline,
                                preload=True)

            # Save using same base name
            base_filename = os.path.splitext(fif_file)[0]
            save_path = os.path.join(save_folder, f"{base_filename}_epo.fif")
            epochs.save(save_path, overwrite=True)
            print(f"✅ Saved epochs: {save_path}\n")

        except Exception as e:
            print(f"❌ Failed to process {fif_file}: {e}\n")


# def create_epochs_from_annotations(raw):
#     """
#      Create epochs from annotations in the raw EEG data.
#     """
#     # 1. Create events from annotations
#     event_id = {'S': 1, 'NS': 2, 'D': 3}  # Map your annotation labels to integers
#     events, event_id_map = mne.events_from_annotations(raw, event_id=event_id)
 
#     # 2. Define epoch parameters
#     sfreq = raw.info['sfreq']
#     tmin = 0.0   # Start of each epoch (0 since annotation is aligned with onset)
#     tmax = (sfreq-1)/sfreq  
#     baseline = None  # You can set to (None, 0) if you want baseline correction

#     # 3. Create epochs
#     epochs = mne.Epochs(raw, events=events, event_id=event_id,
#                         tmin=tmin, tmax=tmax, baseline=baseline,
#                         preload=True)

#     # 4. Save the epochs in .fif format (MNE’s native)
#     epochs_fif_path = 'annotated_epochs.fif'
#     epochs.save(epochs_fif_path, overwrite=True)

#     # Optional: save in EEGLAB .set format if needed
#     eeglab_set_path = 'annotated_epochs.set'
#     mne.export.export_epochs(eeglab_set_path, epochs=epochs, fmt='eeglab', overwrite=True)

#     print("✅ Epochs saved in both .fif and .set format.")
#     return epochs
def load_epochs(file_path: str, format: str = 'fif') -> mne.Epochs:
    """
    Load epochs from a .fif or .set file.

    Parameters:
        file_path (str): Path to the epochs file.
        format (str): 'fif' (MNE format) or 'set' (EEGLAB format)

    Returns:
        mne.Epochs: The loaded epochs object.
    """
    if format == 'fif':
        return mne.read_epochs(file_path, preload=True)
    elif format == 'set':
        return mne.io.read_epochs_eeglab(file_path, preload=True)
    else:
        raise ValueError("Unsupported format. Use 'fif' or 'set'.")

def filter_epochs(epochs: mne.Epochs, keep_labels: List[str]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Filter epochs based on annotation labels and return data and labels.

    Parameters:
        epochs (mne.Epochs): The full epochs object.
        keep_labels (List[str]): List of annotation labels to keep (e.g., ['S', 'NS'])

    Returns:
        Tuple[np.ndarray, np.ndarray]: (data, labels)
            - data shape: (n_epochs, n_channels, n_times)
            - labels: 1D array of integer labels
    """
    filtered_epochs = epochs[keep_labels]
    data = filtered_epochs.get_data()
    labels = filtered_epochs.events[:, -1]  # integer event ID
    return data, labels

In [3]:
create_epochs_from_folder("/home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d", 
                          save_folder="/home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs")

Processing file: eeg51.fif
Opening raw data file /home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg51.fif...
Isotrak not found
    Range : 0 ... 300863 =      0.000 ...  4700.984 secs
Ready.
Reading 0 ... 300863  =      0.000 ...  4700.984 secs...
Used Annotations descriptions: [np.str_('D'), np.str_('NS'), np.str_('S')]
Not setting metadata
4702 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4702 events and 64 original time points ...


/tmp/ipykernel_265/201679259.py:24: RuntimeWarning: This filename (/home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg51.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_filepath, preload=True)


1 bad epochs dropped
✅ Saved epochs: /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg51_epo.fif

Processing file: eeg19.fif
Opening raw data file /home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg19.fif...
Isotrak not found
    Range : 0 ... 576383 =      0.000 ...  9005.984 secs
Ready.
Reading 0 ... 576383  =      0.000 ...  9005.984 secs...
Used Annotations descriptions: [np.str_('D'), np.str_('NS'), np.str_('S')]


/tmp/ipykernel_265/201679259.py:24: RuntimeWarning: This filename (/home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg19.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_filepath, preload=True)


Not setting metadata
9007 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 9007 events and 64 original time points ...
1 bad epochs dropped
✅ Saved epochs: /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg19_epo.fif

Processing file: eeg14.fif
Opening raw data file /home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg14.fif...
Isotrak not found
    Range : 0 ... 238463 =      0.000 ...  3725.984 secs
Ready.
Reading 0 ... 238463  =      0.000 ...  3725.984 secs...
Used Annotations descriptions: [np.str_('D'), np.str_('NS'), np.str_('S')]
Not setting metadata
3727 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 3727 events and 64 original time points ...


...

1 bad epochs dropped
✅ Saved epochs: /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg40_epo.fif

Processing file: eeg69.fif
Opening raw data file /home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg69.fif...
Isotrak not found
    Range : 0 ... 254271 =      0.000 ...  3972.984 secs
Ready.
Reading 0 ... 254271  =      0.000 ...  3972.984 secs...
Used Annotations descriptions: [np.str_('D'), np.str_('NS'), np.str_('S')]
Not setting metadata
3974 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 3974 events and 64 original time points ...


/tmp/ipykernel_265/201679259.py:24: RuntimeWarning: This filename (/home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg69.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_filepath, preload=True)


1 bad epochs dropped
✅ Saved epochs: /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg69_epo.fif

Processing file: eeg31.fif
Opening raw data file /home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg31.fif...
Isotrak not found
    Range : 0 ... 225855 =      0.000 ...  3528.984 secs
Ready.
Reading 0 ... 225855  =      0.000 ...  3528.984 secs...
Used Annotations descriptions: [np.str_('D'), np.str_('NS'), np.str_('S')]
Not setting metadata
3530 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 3530 events and 64 original time points ...
1 bad epochs dropped


/tmp/ipykernel_265/201679259.py:24: RuntimeWarning: This filename (/home/min/Desktop/eegllm/RawEEG_Updated/updated_annotations_with_d/eeg31.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_filepath, preload=True)


✅ Saved epochs: /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg31_epo.fif



In [4]:
# # raw.set_montage("standard_1020") 
# raw_clean, ica = apply_ica(raw, exclude_manual=[0, 1])  # or None
epochs = load_epochs("/home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg51_epo.fif")
# epochs.plot();

Reading /home/min/Desktop/eegllm/RawEEG_Updated/processed_epochs/eeg51_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...     984.38 ms
        0 CTF compensation matrices available
Not setting metadata
4701 matching events found
No baseline correction applied
0 projection items activated


In [5]:
# Filter to all labeled epochs
data, labels = filter_epochs(epochs, keep_labels=['S', 'NS', 'D'])
print("Shape of data:", data.shape)
print("Unique labels:", np.unique(labels))

Shape of data: (4701, 18, 64)
Unique labels: [1 2 3]


In [6]:
# Filter to keep only 'S' and 'NS' labeled epochs
data, labels = filter_epochs(epochs, keep_labels=['S', 'NS'])
print("Shape of data:", data.shape)
print("Unique labels:", np.unique(labels))

Shape of data: (4423, 18, 64)
Unique labels: [1 2]


In [7]:
def count_lables(labels):
    # Assuming `labels` is a 1D NumPy array of event IDs
    unique_labels, counts = np.unique(labels, return_counts=True)

    # Print counts
    for label, count in zip(unique_labels, counts):
        print(f"Label ID {label}: {count} epochs")

count_lables(labels)

Label ID 1: 290 epochs
Label ID 2: 4133 epochs
